In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [ ]:
global sigma
global r
global b

sigma = 10
r = 28
b = 8./3.

In [ ]:
def lorenz_jet(w, t, r, sigma, b):
    wdot = np.zeros(6)
    wdot[0] = sigma * (w[1] - w[0])
    wdot[1] = r * w[0] - w[1] - w[0] * w[2]
    wdot[2] = w[0] * w[1] - b * w[2]
    wdot[3] = sigma * (w[4] - w[3])
    wdot[4] = (r - w[2]) * w[3] - w[4] - w[0] * w[5]
    wdot[5] = w[1] * w[3] + w[0] * w[4] - b * w[5]
    return wdot

In [ ]:
def lyapunov(tmax, r):
    tstep = 1
    x0 = np.array([0.65, -8.464, 2.365])
    v0 = np.array([1.0, 0.0, 0.0])
    w0 = np.concatenate((x0, v0))
    scalefactor = -np.log(np.linalg.norm(v0))
    T = np.array([])
    L = np.array([])

    for time in np.arange(1, tmax + 1, tstep):
        t = np.linspace(time, time + tstep, 2)
        w = odeint(lorenz_jet, w0, t, args=(r, sigma, b))
        lyp = (scalefactor + 0.5 * np.log(np.sum(w[:, 3:6]**2, axis=1))) / t
        T = np.concatenate((T, t))
        L = np.concatenate((L, lyp))
        nm = np.linalg.norm(w[-1, 3:6])**2
        scalefactor += np.log(nm)
        w0 = np.concatenate((w[-1, :3], w[-1, 3:6] / nm))
        
    fig, axs = plt.subplots(1, 1)
    axs.plot(T, L, c = "#A23BEC")
    axs.scatter(T[-1], L[-1], c = 'k')
    axs.annotate(fr'$\mu_{{\text{{max}}}}$ = {round(L[-1], 2)}', xy=(T[-1] - 55, L[-1] - 0.1))
    
    
    axs.set_xlabel("time (s)")
    axs.set_ylabel(r'$\mu_{max}$', rotation = 0)
    axs.set_facecolor("#e1e2e3")
    axs.grid(True)
    axs.set_title(r'$\mu_{\text{max}}$ Estimate')
    return L[-1]

tmax = 400
res = lyapunov(tmax, r)
print(f"Lyapunov exponent for r = {r}: {round(res, 2)}")